import 

In [ ]:
!jupyter server list
import torch

In [ ]:
# test if you have access to cuda
import torch

print("cuda available: ", torch.cuda.is_available())
torch.rand(2, 3).cuda()

# Load des données


In [ ]:
path_train_data = "PATH_TO_DATA/db_train.raw"
path_test_data = "PATH_TO_DATA/db_test.raw"
path_train_label = "PATH_TO_DATA/label_train.txt"

In [ ]:
# load the train data , those are images of size 56x56x3
import numpy as np

X_data = np.fromfile(path_train_data, dtype=np.int16)
Y_train = np.fromfile(path_train_data, dtype=np.int16)

In [ ]:
import torch

X_data = torch.tensor(np.fromfile(path_train_data, dtype="uint8")) / 255
X_data = X_data.reshape(-1, 56, 56, 3)
X_test = torch.tensor(np.fromfile(path_test_data, dtype="uint8")) / 255
X_test = X_test.reshape(-1, 56, 56, 3)
X_test.shape

# load the txt
Y_train = torch.tensor(np.loadtxt(path_train_label, dtype=np.int16))
print(Y_train)

# Visualization des données 


In [ ]:
import cv2

# visualize the 10 first image of X_train and add the respective label
import matplotlib.pyplot as plt

nb_images = 10

fig, axes = plt.subplots(1, nb_images, figsize=(20, 20))

# fig.suptitle("train_test")
for i in range(nb_images):
    axes[i].imshow(X_data[i])
    axes[i].axis("off")
    axes[i].set_title(int(Y_train[i]))

print(" Some sample from the train set ")
# plt.title("train_test")

In [ ]:
# visualize image where the respective label is 0
fig, axes = plt.subplots(1, 10, figsize=(40, 40))
for i in range(10):
    axes[i].imshow(X_data[Y_train == 0][i + 10])
    axes[i].axis("off")
    axes[i].set_title(0)
print("some negative sample from the train set")

In [ ]:
# visualize image where the respective label is 1
fig, axes = plt.subplots(1, 10, figsize=(40, 40))
for i in range(10):
    axes[i].imshow(X_data[Y_train == 1][i + 100])
    axes[i].axis("off")
    axes[i].set_title(1)
print("some positive sample from the train set")

In [ ]:
import cv2

# visualize the 10 first image of X_train
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 10, figsize=(20, 20))
for i in range(10):
    axes[i].imshow(X_test[i])
    axes[i].axis("off")
plt.title("test set")

In [ ]:
# count the number of image for each label
import numpy as np

unique, counts = np.unique(Y_train, return_counts=True)
dict(zip(unique, counts))

for now, you can't guess from images what is the underlying labels , but the 0 label is much more scarce than the 1.
After training a bit and thinking about it, my intuition ( it can be rough if it is not the case) is that some image are AI-generated or not . For example in the picture below, eyes are not aligned. 


In [ ]:
PATH = "image.png"
from IPython.display import Image
from IPython.core.display import HTML

Image(filename=PATH, width=100, height=100)

In [ ]:
PATH = "img_2.png"
from IPython.display import Image
from IPython.core.display import HTML

Image(filename=PATH, width=100, height=100)

# Training


In [ ]:
path_train_data = "PATH_TO_DATA/db_train.raw"
path_test_data = "PATH_TO_DATA/db_test.raw"
path_train_label = "PATH_TO_DATA/label_train.txt"
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# load the train data , those are images of size 56x56x3
import numpy as np
import torch

X_data = np.fromfile(path_train_data, dtype=np.int16)
Y_train = np.fromfile(path_train_data, dtype=np.int16)

X_data = (
    torch.tensor(np.fromfile(path_train_data, dtype="uint8"), dtype=torch.float32) / 255
)

In [ ]:
print("here")
X_data = X_data.reshape(-1, 56, 56, 3)
X_test = (
    torch.tensor(np.fromfile(path_test_data, dtype="uint8"), dtype=torch.float32) / 255
)
X_test = X_test.reshape(-1, 56, 56, 3)
X_test.shape

# load the txt
Y_train = torch.tensor(np.loadtxt(path_train_label, dtype=np.int16), dtype=torch.int64)

In [ ]:
import torch

In [ ]:
from utils import LeNet

input = torch.rand(1, 56, 56, 3)

model = LeNet(2, 0.5)
model(input)

In [ ]:
# split X_data and Y_train into train and validation set
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(
    X_data, Y_train, test_size=0.2, random_state=42
)
# X_train

In [ ]:
# define the dataset
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from utils import CustomImageDataset

training_data = CustomImageDataset(X_train, Y_train)
val_data = CustomImageDataset(X_val, Y_val)
test_data = CustomImageDataset(X_test, np.zeros(X_test.shape[0]))
from torch.utils.data import DataLoader

In [ ]:
# define the optimizer
import torch.optim as optim
import tqdm


# define the training loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    train_loss = 0
    train_acc = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # compute metrics

        train_loss += loss.item()
        train_acc += (pred.argmax(1) == y).type(torch.float).sum().item()
    train_loss /= size
    train_acc /= size
    return train_loss, train_acc

In [ ]:
def valid_epoch(test_loader, model, loss_fn):
    val_loss = 0
    val_acc = 0
    model.eval()
    size = len(test_loader.dataset)
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            val_acc += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= size
    val_acc /= size
    model.train()
    return val_loss, val_acc
    print("End val epoch")

In [ ]:
def run_training(train_dataloader, val_dataloader, model, loss, optimizer, epochs):
    pbar = tqdm(total=epochs, desc="Training")

    log_interval = 1
    train_acc = []
    val_acc_l = []

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}\n-------------------------------")
        train_ep_loss, train_ep_acc = train_loop(
            train_dataloader, model, loss, optimizer
        )

        if epoch % log_interval == 0:
            val_loss, val_acc = valid_epoch(val_dataloader, model, loss)

            print(
                f"Epoch: {epoch}/{epochs}, "
                f"Train Loss: {train_ep_loss:.4f}, "
                f"Train Acc: {1e2*train_ep_acc:.2f}%, "
                f"Val Loss: {val_loss:.4f}, "
                f"Val Acc: {1e2*val_acc:.2f}%"
            )
            train_acc.append(train_ep_acc)
            val_acc_l.append(val_acc)

            pbar.set_postfix({"loss": train_ep_loss})
        pbar.update(1)
    print("Done!")
    pbar.close()
    return model, train_acc, val_acc_l

In [ ]:
# effective trainin
from tqdm.notebook import tqdm
import torch.nn as nn

epochs = 20
model = LeNet(2, 0.5).to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
bs = 256
train_dataloader = DataLoader(training_data, batch_size=bs)
val_dataloader = DataLoader(val_data, batch_size=bs)

test_dataloader = DataLoader(test_data, batch_size=bs)
model, train_acc, val_acc = run_training(
    train_dataloader, val_dataloader, model, loss, optimizer, epochs
)

 ## Interpretation : 
 After a first training With only 20 epochs:
 - Comme la validation vient du même dataset que le dataset d'entrainement, on a pas beaucoup de différence entre le dataset d'entrainement et le dataset de validation 
 - On a un plateau en validation autour de 95% des 5 à 7 epochs. On peut donc s'arrêter à ce nombre d'epoch pour nos expérimentations 
 - La question est de savoir si nos résultats sont bons ou non, calculons l'accuracy sur des modèles "naifs"


In [ ]:
# On verifie que la validation accuracy est meilleur qu'un choix aléatoire
# compute the accuracy of a random model
import numpy as np
from sklearn.metrics import accuracy_score

random_pred = np.random.randint(0, 2, size=len(Y_val))
print(" accuracy si le modèle est aléatoire", accuracy_score(Y_val, random_pred))

# compute the accuracy if we only predict  1
print(
    " accuracy si le modèle prédit toujours 1",
    accuracy_score(Y_val, np.ones(Y_val.shape)),
)
print(" si le modèle prédit toujours 0", accuracy_score(Y_val, np.zeros(Y_val.shape)))

Ici, les bons résultats sont à compenser par le fort déséquilibre des classes. La classe 1 est ultra-majoritaire  et bien que notre modèle est meilleur que le modèle naif,le modèle naif performe déjà beaucoup

In [ ]:
# Compute the recall and precision
from sklearn.metrics import classification_report

y_pred = []
y_true = []
model.eval()
with torch.no_grad():
    for X, y in val_dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        y_pred.extend(pred.argmax(1).cpu().numpy())
        y_true.extend(y.cpu().numpy())
del pred, y
print(classification_report(y_true, y_pred))

Comme attendu le modèle apprend très bien la classe 1 mais arrive quand même à prédire la classe  0 ce qui est pas mal. 
On peut donc calculer le Half Total Error Rate : 
Comme il s'agit d'un taux d'erreurs on le veut aussi bas que possible

In [ ]:
# calcul de la metrique : HTER
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)
# plot it
import matplotlib.pyplot as plt

import seaborn as sns

sns.heatmap(cm, annot=True, fmt="d")
plt.title("Confusion matrix")
plt.xlabel("True label")
plt.ylabel("Predicted label")
plt.show()
HTER = (cm[0][1] / (cm[0][0] + cm[0][1]) + cm[1][0] / (cm[1][0] + cm[1][1])) / 2
print("HTER", HTER)

 # Improve the results :
 Pour augmenter les résultats on pourrait :
 - Changer l'architecture du modèle
 - Ajouter de la Data Augmentation.
 

## Changer l'architecture
On va tester les modèles avec un FeedForward Network classique et une architecture type Resnet

In [ ]:
# create a random FFN network
import torch.nn as nn
import torch.nn.functional as F
from utils import FFN

input = torch.rand(1, 56, 56, 3)

model = FFN(2, dropout=0.2)
model(input)

In [ ]:
# effective training
model = FFN(2, dropout=0.2).to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
bs = 256
train_dataloader = DataLoader(training_data, batch_size=bs)
val_dataloader = DataLoader(val_data, batch_size=bs)
epochs = 20
model, train_acc, val_acc = run_training(
    train_dataloader, val_dataloader, model, loss, optimizer, epochs
)

Le rajout d'un modèle de type FEED Forward network n'a pas beaucoup aidé, indiquant que les informations locales qu'extrait le CNN semblent plus efficace.

In [ ]:
import torch.nn as nn
from utils import ResidualBlock


class ResNet(nn.Module):
    def __init__(self, layers, nb_classes, block=ResidualBlock):
        super(ResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer0 = self._make_layer(block, 16, layers[0])
        self.layer1 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, nb_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes  # Update inplanes before subsequent blocks
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)  # Flatten keeping batch size intact
        x = self.fc(x)
        return x


input = torch.rand(1, 56, 56, 3)

model = ResNet([2, 2, 2], 2)
model(input)

In [ ]:
# effective training
model = ResNet([2, 2, 2], 2).to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
bs = 256
train_dataloader = DataLoader(training_data, batch_size=bs)
val_dataloader = DataLoader(val_data, batch_size=bs)
epochs = 5
model, train_acc, val_acc = run_training(
    train_dataloader, val_dataloader, model, loss, optimizer, epochs
)

This model has almost the same validation results has the initial CNN but overfit on training test and is more slow,  the baseline should be kept I guess

## Transformation
We add here some Data Augmentation

In [ ]:
import torchvision.transforms as transforms
from utils import PermuteChannels, PermuteChannelsinv

transform_train = transforms.Compose(
    [
        PermuteChannels(),  # Permutes (H, W, C) -> (C, H, W) if needed
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        PermuteChannelsinv(),
    ]
)

input = transform_train(X_data[0])

In [ ]:
# add the transform to the dataset
training_data_transform = CustomImageDataset(
    X_train, Y_train, transform=transform_train
)
val_data_transform = CustomImageDataset(X_val, Y_val, transform=None)

# effective training

epochs = 20
model = LeNet(2, 0.5).to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
bs = 256
train_dataloader = DataLoader(training_data_transform, batch_size=bs, num_workers=4)
val_dataloader = DataLoader(val_data_transform, batch_size=bs, num_workers=4)

test_dataloader = DataLoader(test_data, batch_size=bs)
model_transform, train_acc, val_acc = run_training(
    train_dataloader, val_dataloader, model, loss, optimizer, epochs
)

On a gagné environ 0.7% d'accuracy avec nos transformations. Cela semble indiqué qu'elles pourraient améliorer nos résultats

## Conclusion des expérimentations: 
- Nous avons testé 3 modèles: un LetNet qui correspond à des CNN, un FeedForward classqiue  et une architecture type Resnet.  le Resnet et le CNN donnait des résultats similaires mais nous gardons LeNet car son architecture était plus simple 
- Concernant la Data Augmentation ils améliorent légérement le résultat du modèle. Nous pouvons le garder car il conserve un temps de calcul raisonnable

## Calcul sur la métrique de test

In [ ]:
model = model_transform

In [ ]:
model.eval()
# predict the test set
y_pred = []
y_true = []
with torch.no_grad():
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        y_pred.extend(pred.argmax(1).cpu().numpy())
        y_true.extend(y.cpu().numpy())

In [ ]:
# save the prediction, save as a txt format
np.savetxt("prediction.txt", y_pred)

In [ ]:
# load the prediction just in case
y_load = np.loadtxt("prediction.txt")